In [1]:
%load_ext lab_black

In [15]:
import json
import requests
import yaml

url = "https://ckan.opendata.swiss/api/3/action/package_show?id=median-vermogen-steuerpflichtiger-naturlicher-personen-nach-jahr-steuertarif-und-stadtquartier3"

resp = requests.get(url)
resp.content[:500]

b'{"help": "https://ckan.opendata.swiss/api/3/action/help_show?name=package_show", "success": true, "result": {"license_title": null, "maintainer": "Open Data Z\\u00fcrich", "issued": "2016-02-01T00:00:00", "title_for_slug": "median-vermogen-steuerpflichtiger-naturlicher-personen-nach-jahr-steuertarif-und-stadtquartier3", "qualified_relations": [], "private": false, "maintainer_email": "opendata@zuerich.ch", "num_tags": 6, "contact_points": [{"email": "opendata@zuerich.ch", "name": "Open Data Z\\u00'

In [16]:
resp.status_code

200

In [17]:
resp.encoding

'utf-8'

In [24]:
json_personal_asset = resp.json()
personal_asset_url = json_personal_asset["result"]["resources"][0]["url"]

In [25]:
import pandas as pd

zurich_personal_asset = pd.read_csv(personal_asset_url)

In [29]:
zurich_personal_asset_ytd = zurich_personal_asset.loc[
    zurich_personal_asset["StichtagDatJahr"]
    == zurich_personal_asset["StichtagDatJahr"].max()
]
zurich_personal_asset_ytd = zurich_personal_asset_ytd.dropna()
zurich_personal_asset_ytd.head(10)
# Tax year (technically: key date date year)	Year for which the tax was claimed (integer).
# City quarter (sort) (technically: QuarSort)	ID for the city district at the current residential address (integer).
# City quarter (code) (technical: QuarCd)	ID for the city district at the current residential address (string).
# City Quarter (Lang) (technically: QuarLang)	City district name of the current residential address (string).
# Tax tariff sort (technically: tax tariff sort)	Code of the tax rate according to which taxes are calculated (integer).
# Tax tariff code (technically: TaxTarifCd)	Code of the tax rate according to which taxes are calculated (string).
# Tax tariff (technically: SteuerTarifLang)	Name of the tax rate according to which the taxes are calculated (string).
# Median value (technically: SteuerVermoegen_p50)	Median value of taxable assets of natural persons in Zurich. In thousand francs (integers).
# P25 value (technically: SteuerVermoegen_p25)	25% quantile value of the taxable assets of natural persons in Zurich. In thousand francs (integers).
# P75 value (technically: SteuerVermoegen_p75)	75% quantile value of the taxable assets of natural persons in Zurich. In thousand francs (integers).

# https://data.stadt-zuerich.ch/dataset/fd_median_vermoegen_quartier_od1004

,StichtagDatJahr,QuarSort,QuarCd,QuarLang,SteuerTarifSort,SteuerTarifCd,SteuerTarifLang,SteuerVermoegen_p50,SteuerVermoegen_p25,SteuerVermoegen_p75
2142,2020,11,11,Rathaus,0,0,Grundtarif,57.0,3.0,319.0
2143,2020,11,11,Rathaus,1,1,Verheiratetentarif,426.5,96.0,1530.0
2144,2020,11,11,Rathaus,2,2,Einelternfamilientarif,110.0,9.5,563.0
2145,2020,12,12,Hochschulen,0,0,Grundtarif,62.0,4.0,326.0
2146,2020,12,12,Hochschulen,1,1,Verheiratetentarif,774.0,80.0,1639.0
2148,2020,13,13,Lindenhof,0,0,Grundtarif,85.0,8.0,461.0
2149,2020,13,13,Lindenhof,1,1,Verheiratetentarif,663.5,222.0,1988.0
2151,2020,14,14,City,0,0,Grundtarif,22.0,1.0,164.5
2152,2020,14,14,City,1,1,Verheiratetentarif,262.0,55.0,774.0
2154,2020,21,21,Wollishofen,0,0,Grundtarif,49.0,5.0,216.0


In [31]:
zurich_personal_asset_ytd_median = (
    zurich_personal_asset_ytd.groupby(["QuarLang", "SteuerTarifLang"])
    .agg("median")
    .reset_index()
)

tax_group_dict = {
    "Einelternfamilientarif": "Single-parent",
    "Grundtarif": "Basic single",
    "Verheiratetentarif": "Married person",
}

zurich_personal_asset_ytd_median = zurich_personal_asset_ytd_median.replace(
    {"SteuerTarifLang": tax_group_dict}
)

zurich_personal_asset_ytd_median

,QuarLang,SteuerTarifLang,StichtagDatJahr,QuarSort,QuarCd,SteuerTarifSort,SteuerTarifCd,SteuerVermoegen_p50,SteuerVermoegen_p25,SteuerVermoegen_p75
0,Affoltern,Single-parent,2020.0,111.0,111.0,2.0,2.0,4.0,0.0,63.5
1,Affoltern,Basic single,2020.0,111.0,111.0,0.0,0.0,20.0,0.0,119.0
2,Affoltern,Married person,2020.0,111.0,111.0,1.0,1.0,93.0,4.0,334.0
3,Albisrieden,Single-parent,2020.0,91.0,91.0,2.0,2.0,49.0,3.0,198.0
4,Albisrieden,Basic single,2020.0,91.0,91.0,0.0,0.0,45.0,4.0,184.0
...,...,...,...,...,...,...,...,...,...,...
94,Witikon,Basic single,2020.0,74.0,74.0,0.0,0.0,81.0,9.0,444.0
95,Witikon,Married person,2020.0,74.0,74.0,1.0,1.0,405.5,96.0,1292.0
96,Wollishofen,Single-parent,2020.0,21.0,21.0,2.0,2.0,45.0,2.0,218.0
97,Wollishofen,Basic single,2020.0,21.0,21.0,0.0,0.0,49.0,5.0,216.0
